In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import re
import time
from typing import List, Any, Union
import pandas as pd
import multiprocessing as mp
import copy as cp

In [ ]:
def reFormatTrainData(data):
    format_data = {}
    for line in data:
        line = re.sub(r"(?<=\d)\n", "", line)
        pattern = re.compile("(?<=\d)\t(?=\d)")
        line = re.split(pattern, line)
        format_data[line[0]] = line[1:]
    return format_data

def reFormatTestData(data):
    format_data = []
    for line in data:
        line = re.sub(r"(?<=\d)\n", "", line)
        pattern = re.compile("(?<=\d)\t(?=\d)")
        line = re.split(pattern, line)
        line[0] = int(line[0])
        format_data.append(line)
    return format_data

# read train data
file_dir = '../input/comp90051-2020-sem2-proj1/'
file_name = "train.txt"
with open(os.path.join(file_dir, file_name)) as f:
    train_set = f.readlines()
    train_set = reFormatTrainData(train_set)

# read test data
test_name = "test-public.txt"
with open(os.path.join(file_dir, test_name)) as f:
    f.readline()
    test_set = f.readlines()
    test_set = reFormatTestData(test_set)
    
print(len(train_set))
print(len(test_set))

In [ ]:
# Combin in and out
def getNeighbors(node, graph):
    followers = getFollowers(node, graph)
    followees = getFollowees(node, graph)
    neighbors = followers.union(followees)
    return neighbors

# out
def getFollowers(node, graph):
    followers = set()
    try:
        for item in graph[node]:
            followers.add(item)
        return followers
    except:
        return followers

# in
def getFollowees(node, graph):
    followees = set()
    for key, value in graph.items():
        if node in value:
            followees.add(key)
    return followees

# Common Neighbors in Level 1
def commonNeighborsLevel1(a, b, graph):
    neighbors_a = getNeighbors(a, graph)
    neighbors_b = getNeighbors(b, graph)
    return len(neighbors_a.intersection(neighbors_b))

# Common Neighbors in Level 2
def commonNeighborsLevel2(a, b, graph):
    score = 0
    followers_a = getFollowers(a, graph)
    if followers_a:
        for node in followers_a:
            score += commonNeighborsLevel1(b, node, graph)
        return float(score / len(followers_a))
    else:
        return 0

# Feature Extraction
def getFeatures(record: list) -> list:
    node= [record[0],len(getFollowees(record[1],train_set)),len(getFollowers(record[1],train_set)),len(getFollowees(record[-1],train_set)),len(getFollowers(record[-1],train_set)),commonNeighborsLevel1(record[1], record[-1], train_set)]
    return node

In [ ]:
pool = mp.Pool(mp.cpu_count())
Common_neighbors2 = pool.map(getFeatures, test_set[0:200])
pool.close()
pool.join()